### 基于CrawlSpider的全站爬取
- CrawlSpider是爬虫类Spider的一个子类
- 使用流程：
    - 1.创建一个基于CrawlSpider的爬虫文件：scrapy genspider -t crawl 爬虫文件名称 www.xxx.com
    - 2.构造链接提取器和规则解析器
        - 链接提取器：
            - 作用：根据指定规则进行指定链接提取
            - 提取规则：allow='正则表达式'
        - 规则解析器：
            - 作用：获取链接提取器取到的链接，然后对其进行请求发送，根据指定规则对请求到的页面源码数据进行解析
        - follow=True：将链接提取器继续作用到链接提取器提取出的URL对应的页面中

### 分布式
- 什么是分布式？
    - 基于多台计算机组建一个分布式集群，然后使集群中的每一台计算机执行同一组程序，让它们对同一个网站的数据进行分布爬取
- 为什么使用分布式爬虫？
    - 提升爬取数据效率
- 如何实现分布式爬虫？
    - 基于scrapy+redis实现分布式
        - scrapy结合scrapy-redis组件实现分布式
- 原生的scrapy框架无法实现分布式
    - 调度器无法被集群共享
    - 管道无法被共享
- scrpay-redis组件作用：提供可以被共享的调度器和管道
- 环境安装：
    - redis
    - pip install scrapy-redis
- 流程：
    - 1.创建一个工程
    - 2.创建一个爬虫文件：基于CrawlSpider的爬虫文件
        - 修改当前爬虫文件
            - 导包：from scrapy_redis.spiders import RedisCrawlSpider
            - 将start_urls替换成redis_key = 'XXX'，共享调度器的队列名称
            - 编写爬虫类爬取数据操作
        - 修改settings.py文件
            - 开启可以被共享的管道：ITEM_PIPELINES = { 'scrapy_redis.pipelines.RedisPipeline': 400, }
            - 去重容器：DUPEFILTER_CLASS ="scrapy_redis.dupefilter.RFPDupeFilter"
            - 指定共享调度器：SCHEDULER = "scrapy_redis.scheduler.Scheduler"
            - 实现增量式爬取：SCHEDULER_PERSIST=True
            - 指定redis服务：REDIS_HOST = '本机ip'，REDIS_PORT = 6379
        - redis的配置文件（redis.windows.conf）进行修改：
            - bind 127.0.0.1：进行注释
            - protected-mode no：分布式集群中的其他计算机可以进行写入数据
            - 我们使用redis版本：2.8，可以不进行设计保护模式。原因如下：Redis从3.2开始加强安全管理，如果redis没有设置密码，那么redis客户端只能从本地进行访问，如果是从其他机器连接过来访问的，就会报错误。
        - 携带配置文件启动redis服务：redis-server .\redis.windows.conf
        - 启动redis客户端
        - 执行当前工程：
            - 1. 进入爬虫文件对应目录下，执行命令：scrapy runspider xxx.py
            - 2. 向调度队列中扔入一个起始url：
                - 队列在哪？队列在redis中
                - lpush fbsQueue www.xxx.com

### 增量式爬虫
- 概念：监测网站数据更新情况
- 核心：去重
- 深度爬取类型的网站中需要对详情页的url进行记录和检测
    - 记录：将爬取过的详情页的url进行记录保存；存储到redis的set中
    - 检测：若对某一个详情页的url发起请求之前，先要去记录表中进行查看，该url是否存在。若存在，则被爬取过

- 非深度爬取类型的网站：
    - 名词：数据指纹。一组数据的唯一标识
    - 使用md5等方式实现


